# ノイズデータ作成
- Narrow instrumental artifacts + gapなし非定常ノイズデータを生成

In [1]:
DEBUG = True
#DEBUG = False

OUTDIR = "/volume/kaggle/g2net2/pyfstat_output/tmp_noise"
#OUTDIR = "/volume/kaggle/g2net2/pyfstat_output/ex007_generating_noise_non_stationary"
IS_DEL_OUTDIR = True

#TMP_OUTDIR = "my_generating_noise_1"
TMP_OUTDIR = "/volume/kaggle/g2net2/pyfstat_output/PyFstat_tmp_files/my_generating_noise_2"

N_LEN = 3
N_LEN = 1

In [2]:
# Parameters
TMP_OUTDIR = "/volume/kaggle/g2net2/pyfstat_output/PyFstat_tmp_files/my_generating_noise_2"
OUTDIR = "/volume/kaggle/g2net2/pyfstat_output/ex009_generating_noise_artifacts_n_len3"
N_LEN = 3
IS_DEL_OUTDIR = True
DEBUG = False


In [3]:
#!pip install h5py
#!pip install pandas
import h5py
import pandas as pd

In [4]:
import os
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy import stats

import pyfstat
from pyfstat.utils import get_sft_as_arrays

# Local module to simplify plotting
import tutorial_utils

logger = pyfstat.set_up_logger(label=TMP_OUTDIR, log_level="INFO")

%matplotlib inline

22-12-24 09:42:16.795 pyfstat INFO    : Running PyFstat version 1.18.1+84.5ded4a67.clean


22-12-24 09:42:16.862 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactive 'Agg' backend.


In [5]:
# ================================================
# 生成したhdf5ファイルロードできるか確認
# ================================================
import h5py
from pathlib import Path

def load_hdf5(h_path):
    """hdf5ファイルロード"""
    with h5py.File(h_path, 'r') as f:
        g = f[file_id]

        # [:]を最後につけないと取り出せないので注意
        freq = g['frequency_Hz'][:]  # shape: (360,) 各振動数

        h1_a = g["H1"]['SFTs'][:]
        h1_t = g["H1"]['timestamps_GPS'][:]

        l1_a = g["L1"]['SFTs'][:]
        l1_t = g["L1"]['timestamps_GPS'][:]
    
    return freq, h1_a, l1_a, h1_t, l1_t

# gen_artifacts_noise_detectors

In [6]:
def gen_artifacts_noise_detectors(out_path, file_id, sqrtSX, h0, F0, tstart, detectors):
    """detectors指定してartifactsノイズを生成
    gap、非定常との併用できなかった
    """
    # Setup Writer    
    writer_kwargs = {
        "label": "single_detector_spectral_line",
        "outdir": TMP_OUTDIR,
        "tstart": tstart,  # 観測開始時刻 [GPS時刻]
        "duration": 30 * 4 * 24 * 3600,  #5 * 86400,  # 継続時間[秒］。4ヶ月
        "detectors": detectors,  # シミュレーションする検出器(ここではLIGOハンフォード)
        "F0": F0,  # 生成するバンドの中心周波数 [Hz]. [F0 - Band/2, F0 + Band/2]
        "Band": 0.5,  #1.0,  # F0付近の周波数帯域幅 [Hz]               
        "sqrtSX": sqrtSX,  # 雑音の片側振幅スペクトル密度
        "Tsft": 1800,  # フーリエ変換の継続時間
        "SFTWindowType": "tukey",
        "SFTWindowBeta": 0.01,

        #"h0": 3.2e-25, # 10ぐらいになる
        "h0": h0,
        
        #"phi": 1.3,  # スペクトル線の初期位相
        "phi": 2*np.pi*np.random.rand(),
    }
    #print(sqrtSX, writer_kwargs["h0"])
    
    # =================================================
    # めちゃくちゃ中間ファイル作られるから実行前にきれいにする
    # =================================================
    if os.path.exists(TMP_OUTDIR):
        shutil.rmtree(TMP_OUTDIR)
        os.mkdir(TMP_OUTDIR)
        
    # =================================================
    # データ生成
    # =================================================
    writer = pyfstat.LineWriter(**writer_kwargs)  # LineWriterじゃないと線かけない
    writer.make_data()

    frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)  # SFTデータをnumpyの配列に読み込み
    
    return frequency, timestamps, fourier_data


def gen_artifacts_noise(out_path, file_id, F0, sqrtSX=1e-24, h1_h0=1e-25, l1_h0=1e-25):
    """artifactsノイズデータを生成してhdf5に保存する
    gap、非定常との併用できなかった
    """
    for _ in range(10):  # segmentアリだと4096未満になるときあるので対策
        
        # 関係ないと思うが一応時刻もランダムに変えとく
        tstart = np.random.randint(1238166018,1248724262)  # 観測開始時刻 [GPS時刻]

        # h1とl1それぞれでデータ生成
        frequency, timestamps_h1, fourier_data_h1 = gen_artifacts_noise_detectors(out_path, file_id, sqrtSX, h1_h0, F0, tstart, "H1")
        frequency, timestamps_l1, fourier_data_l1 = gen_artifacts_noise_detectors(out_path, file_id, sqrtSX, l1_h0, F0, tstart, "L1")

        # =================================================
        # 対象の周波数帯域の 0.2 の範囲だけスライス
        # =================================================
        # https://www.kaggle.com/competitions/g2net-detecting-continuous-gravitational-waves/discussion/347052#1987496
        f_roll = np.random.uniform(low=0.01, high=0.19)
        first_index = np.argmin( np.abs( frequency - (F0 - f_roll) ) )
        last_index = np.argmin( np.abs( frequency - (F0 + (2.0 - f_roll) ) ) )
        frequency = frequency[first_index:last_index+1]
        h1_amp = fourier_data_h1["H1"][first_index:last_index + 1, :]
        l1_amp = fourier_data_l1["L1"][first_index:last_index + 1, :]

        # 360の長さまでにしておく（361になるときがあるので）
        frequency = frequency[:360]
        h1_amp = h1_amp[:360,:]
        l1_amp = l1_amp[:360,:]
        
        if (len(timestamps_h1["H1"]) >= 4096) and (len(timestamps_l1["L1"]) >= 4096):
            break

    # =================================================
    # HDF5ファイルの作成
    # =================================================
    if (len(timestamps_h1["H1"]) >= 4096) and (len(timestamps_l1["L1"]) >= 4096):
                                                           
        with h5py.File(f"{out_path}", "w") as h5:
            g = h5.create_group(str(file_id))
            g.create_dataset('frequency_Hz', data=frequency)

            g_h = g.create_group("H1")
            g_h.create_dataset('SFTs', data=h1_amp)
            g_h.create_dataset('timestamps_GPS', data=timestamps_h1["H1"])

            g_l = g.create_group("L1")
            g_l.create_dataset('SFTs', data=l1_amp)
            g_l.create_dataset('timestamps_GPS', data=timestamps_l1["L1"])
    
    return frequency, timestamps_h1["H1"], timestamps_l1["L1"], h1_amp, l1_amp

In [7]:
debug = False
#debug = True
if debug:
    sqrtSX_base = 3.2e-24  # h1,l1 min = 0.3程度になる
    
    #sqrtSX = 2 * sqrtSX_base  # h1,l1 min = 1.2程度になる
    sqrtSX = 3 * sqrtSX_base  # h1,l1 min = 3.7程度になる
    #sqrtSX = 5 * sqrtSX_base  # h1,l1 min = 10程度になる
    #sqrtSX = 10 * sqrtSX_base  # h1,l1 min = 40程度になる
    
    #h1_h0 = sqrtSX_base*10  # 80000ぐらいになる
    h1_h0 = sqrtSX_base*6.5  # 35000ぐらいになる
    #h1_h0 = sqrtSX_base*3  # 7000ぐらいになる
    #h1_h0 = sqrtSX_base*1.2  # 1200ぐらいになる
    #h1_h0 = sqrtSX_base/1.2  # 500ぐらいになる
    #h1_h0 = sqrtSX_base/2  # 200ぐらいになる
    #h1_h0 = sqrtSX_base/5  # 35ぐらいになる
    #h1_h0 = sqrtSX_base/50  # 1.8ぐらいになる
    
    frequency, h1_ts, l1_ts, h1_amp, l1_amp = gen_artifacts_noise(f"./tmp/tmp.hdf5", "tmp", 
                                                                  56, 
                                                                  sqrtSX=sqrtSX, 
                                                                  h1_h0=h1_h0, 
                                                                  l1_h0=0
                                                                 )
    #from IPython.display import clear_output  # セルの出力削除（ログあると重すぎるので消す)
    #clear_output()

In [8]:
if debug:
    
    h_path = f"./tmp/tmp.hdf5"
    file_id = Path(h_path).stem

    freq, h1_a, l1_a, h1_t, l1_t = load_hdf5(h_path)
    print(freq.shape)
    print(h1_a.shape, l1_a.shape)
    print(h1_t.shape, l1_t.shape)

    h1_ts_gaps = h1_t[1:] - h1_t[:-1] - 1800
    print("h1 sum gap:",  round(sum(h1_ts_gaps) / (24*3600), 2) , "days")
    #print(list(h1_ts_gaps))

    l1_ts_gaps = l1_t[1:] - l1_t[:-1] - 1800
    print("l1 sum gap:",  round(sum(l1_ts_gaps) / (24*3600), 2) , "days")
    #print(list(l1_ts_gaps))

    fig_h1, axs = tutorial_utils.plot_real_imag_spectrograms(h1_t, freq, h1_a)
    fig_h1, axs = tutorial_utils.plot_real_imag_spectrograms(l1_t, freq, l1_a)
    #fig_h1, axs = tutorial_utils.plot_real_imag_spectrograms_with_gaps(h1_t, freq, h1_a, 1800)
    #fig_h1, axs = tutorial_utils.plot_real_imag_spectrograms_with_gaps(l1_t, freq, l1_a, 1800)

In [9]:
def show_1img_2ch(p_h1, p_l1, ts_h1, ts_l1, f, y, is_show, save_path="", figsize=(18, 5)):
    
    #p_h1, p_l1 = p_h1 * 255, p_l1 * 255
    
    plt.figure(figsize=figsize)

    #print(p_h1.max(), p_l1.max())
    #print(ts_h1, ts_l1, f.max(), f.min())
    
    if figsize[0] > figsize[1]:
        plt.subplot(2, 1, 1)
    else:
        plt.subplot(1, 2, 1)
    plt.title(f'H1 Spectrogram y:{y}')
    plt.xlabel('time')
    plt.ylabel('frequency')
    plt.imshow(p_h1)  # , extent=[ts_h1.min(), ts_l1.max(), f.max(), f.min()]
    plt.colorbar()

    if figsize[0] > figsize[1]:
        plt.subplot(2, 1, 2)
    else:
        plt.subplot(1, 2, 2)
    plt.title(f'L1 Spectrogram y:{y}')
    plt.xlabel('time')
    plt.ylabel('frequency')
    plt.imshow(p_l1)
    plt.colorbar()

    if save_path != "":
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    
    if is_show:
        plt.tight_layout()
        plt.show()
    plt.clf()  # メモリ解放
    plt.close()

def get_hdf5(h_path, y, 
             is_gap=False,
             is_p_mean=False,
             is_show=False, 
             save_dir="", 
             f_coff=1e-22):
    file_id = Path(h_path).stem
    with h5py.File(h_path, 'r') as f:
        g = f[file_id]
        
        f = g['frequency_Hz'][:]  # Dataset型なので[:]付けたらnpになる
        
        ts_h1 = g["H1"]['timestamps_GPS'][:]  # Dataset型なので[:]付けたらnpになる
        ts_l1 = g["L1"]['timestamps_GPS'][:]  # Dataset型なので[:]付けたらnpになる
        #ts_h1 = [ts2time(ts) for ts in ts_h1]
        #ts_l1 = [ts2time(ts) for ts in ts_l1]
        
        a_h1 = g["H1"]['SFTs'][:]  # Dataset型なので[:]付けたらnpになる
        a_l1 = g["L1"]['SFTs'][:]  # Dataset型なので[:]付けたらnpになる
        
        a_h1_norm = a_h1 / f_coff
        p_h1 = a_h1_norm.real**2 + a_h1_norm.imag**2
        #p_h1 /= np.mean(p_h1)  # normalize
        
        a_l1_norm = a_l1 / f_coff
        p_l1 = a_l1_norm.real**2 + a_l1_norm.imag**2
        #p_l1 /= np.mean(p_l1)  # normalize
        
        if is_p_mean:
            # パワースペクトルはtimestepsごとに平均化. compress 4096 -> 128
            p_h1 = p_h1[:, :4096]
            p_l1 = p_l1[:, :4096]
            p_h1 = np.mean(p_h1.reshape(360, 128, 4096//128), axis=2)
            p_l1 = np.mean(p_l1.reshape(360, 128, 4096//128), axis=2)
            figsize = (10, 10)
        else:
            figsize = (18, 5)
        
        if is_show or save_dir != "":
            show_1img_2ch(p_h1, p_l1, ts_h1, ts_l1, f, y, is_show,
                          save_path=f"{save_dir}/{Path(h_path).stem}_{str(y)}.png", 
                          figsize=figsize,
                         )
            print("h1,l1 max:", p_h1.max(), p_l1.max())
            print("h1,l1 min:", p_h1.min(), p_l1.min())
            print("h1,l1 mean:", p_h1.mean(), p_l1.mean())
            print("h1,l1 std:", p_h1.std(), p_l1.std())
        
        return file_id, f, ts_h1, ts_l1, a_h1, a_l1, p_h1, p_l1

In [10]:
if debug:
    file_id, f, ts_h1, ts_l1, a_h1, a_l1, p_h1, p_l1 = get_hdf5(f"./tmp/tmp.hdf5", 0, 
                                                                is_p_mean=True,
                                                                is_show=True,
                                                                save_dir="tmp",
                                                               )
    print(p_h1.max(), p_l1.max())

In [11]:
#=====

# Main

In [12]:
# ================================================
# hdf5ファイル生成
# ================================================
def main(debug=True):
            
    def _gen_artifacts_noise(F0, sqrtSX, h1_h0, l1_h0, n_len, start_id=0):
        for i in tqdm(range(n_len)):
            _sqrtSX = '{:.3g}'.format(sqrtSX)
            _h1_h0 = '{:.3g}'.format(h1_h0)
            _l1_h0 = '{:.3g}'.format(l1_h0)
            file_id = f"{start_id + i}_sqrtSX{_sqrtSX}_F0{str(round(F0,2))}_h1h0{_h1_h0}_l1h0{_l1_h0}"
            out_path = f"{OUTDIR}/{file_id}.hdf5"
            #print(out_path)

            try:
                for _ in range(10):
                    frequency, h1_ts, l1_ts, h1_amp, l1_amp = gen_artifacts_noise(out_path, str(file_id), F0, 
                                                                                  sqrtSX=sqrtSX, 
                                                                                  h1_h0=h1_h0, 
                                                                                  l1_h0=l1_h0)
                    # 一応5760未満になるときの対策しとく
                    if (len(h1_ts) >= 5760) and (len(l1_ts) >= 5760):
                        break
            except:
                print("Error")
        return i+1
    
    if debug:
        n_len = 1 
    else:
        n_len = N_LEN

    sqrtSX_base = 3.2e-24  # h1,l1 min = 0.3程度になる

    _id = 0
    
    ### h1
    _id += _gen_artifacts_noise(302.3, 2*sqrtSX_base, sqrtSX_base*6.5, 0, n_len, start_id=_id)
    for f0 in [
        56.935,
        52,
        55,
        53.585,
        55.9,
        56.898,
        56.95,
        57.975,
        54.65,
        65.29,
        99.975,
        57.88,
        61.89,
        59.875,
        65.825,
        64.825,
        60.61,
        92.91,
        71.128,
        66.7,
        66.7,
        67.875,
        67.9,
        71,
        69.87,
        70.01,
        71.628,
        73.625,
        0,
        75,
        82.448,
        82.448,
        0.098,
        115.879,
        82,
        85,
        82,
        93.827,
        402.99,
        398.99,
        394.5,
        104.14,
        104.285,
    ]:
        _sqrtSX = np.random.uniform(low=1.5, high=3) * sqrtSX_base
        _h1_h0 = np.random.uniform(low=1/10, high=1.2) * sqrtSX_base
        _id += _gen_artifacts_noise(f0, _sqrtSX, _h1_h0, 0, n_len, start_id=_id)
        
        if debug:
            break
    
    ### l1
    _id += _gen_artifacts_noise(312.04, 2.8*sqrtSX_base, 0, sqrtSX_base*4.5, n_len, start_id=_id)
    _id += _gen_artifacts_noise(312.165, 2.8*sqrtSX_base, 0, sqrtSX_base*4.5, n_len, start_id=_id)
    _id += _gen_artifacts_noise(306.22, 2.8*sqrtSX_base, 0, sqrtSX_base*5, n_len, start_id=_id)
    _id += _gen_artifacts_noise(311.04, 3*sqrtSX_base, 0, sqrtSX_base*4.5, n_len, start_id=_id)
    for f0 in [
        100,
        305.35,
        431.9,
        85.535,
    ]:
        _sqrtSX = np.random.uniform(low=1, high=2) * sqrtSX_base
        _l1_h0 = np.random.uniform(low=1/10, high=0.5) * sqrtSX_base
        _id += _gen_artifacts_noise(f0, _sqrtSX, 0, _l1_h0, n_len, start_id=_id)
        
        if debug:
            break
    
if __name__ == '__main__':

    if IS_DEL_OUTDIR:
        if os.path.exists(OUTDIR):
            shutil.rmtree(OUTDIR)
    os.makedirs(f"{OUTDIR}", exist_ok=True)
        
    main(debug=DEBUG)
    

from IPython.display import clear_output  # セルの出力削除（ログあると重すぎるので消す)
clear_output()

In [13]:
_df = pd.DataFrame({"id": [Path(p).stem for p in sorted(glob.glob(f"{OUTDIR}/*h1h0*.hdf5"))], "target": 0})
_df.to_csv(OUTDIR + "/df.csv", index=False)

fmins, fmaxs = [], []
h1_hsizes, h1_wsizes = [], []
l1_hsizes, l1_wsizes = [], []
h1_amp_mins, h1_amp_maxs = [], []
l1_amp_mins, l1_amp_maxs = [], []
h1_tmins, h1_tmaxs = [], []
l1_tmins, l1_tmaxs = [], []
h1_max_intevals, l1_max_intevals = [], []
for _id in tqdm( _df["id"].to_numpy() ):
    h_path = f"{OUTDIR}/{_id}.hdf5"
    file_id = Path(h_path).stem
    freq, h1_a, l1_a, h1_t, l1_t = load_hdf5(h_path)
    
    fmins.append(freq.min())
    fmaxs.append(freq.max())
    
    h1_hsizes.append(h1_a.shape[0])
    h1_wsizes.append(h1_a.shape[1])
    
    l1_hsizes.append(l1_a.shape[0])
    l1_wsizes.append(l1_a.shape[1])
    
    h1_amp_mins.append(h1_a.min())
    h1_amp_maxs.append(h1_a.max())
    
    l1_amp_mins.append(l1_a.min())
    l1_amp_maxs.append(l1_a.max())

    h1_tmins.append(h1_t.min())
    h1_tmaxs.append(h1_t.max())
    
    l1_tmins.append(l1_t.min())
    l1_tmaxs.append(l1_t.max())
    
    h1_max_inteval = np.max(h1_t[1:]-h1_t[0:-1])
    l1_max_inteval = np.max(l1_t[1:]-l1_t[0:-1])
    h1_max_intevals.append(h1_max_inteval)
    l1_max_intevals.append(l1_max_inteval)

_df["freq min"] = fmins
_df["freq max"] = fmaxs
_df["H1 SFTs h"] = h1_hsizes
_df["H1 SFTs w"] = h1_wsizes
_df["H1 time min"] = h1_tmins
_df["H1 time max"] = h1_tmaxs
_df["H1 max inteval"] = h1_max_intevals
_df["L1 SFTs h"] = l1_hsizes
_df["L1 SFTs w"] = l1_wsizes
_df["L1 time min"] = l1_tmins
_df["L1 time max"] = l1_tmaxs
_df["L1 max inteval"] = l1_max_intevals
_df["freq_min//10"] = _df["freq min"]//10
_df["freq_min//10_target"] = _df["freq_min//10"].astype(str) + "_" + _df["target"].astype(str)
    
_df.to_csv(OUTDIR + "/df_summary.csv", index=False)
_df

  0%|          | 0/150 [00:00<?, ?it/s]

,id,target,freq min,freq max,H1 SFTs h,H1 SFTs w,H1 time min,H1 time max,H1 max inteval,L1 SFTs h,L1 SFTs w,L1 time min,L1 time max,L1 max inteval,freq_min//10,freq_min//10_target
0,0_sqrtSX6.4e-24_F0302.3_h1h02.08e-23_l1h00,0,302.138333,302.337778,360,5760,1241448058,1251814258,1800,360,5760,1241448058,1251814258,1800,30.0,30.0_0
1,102_sqrtSX5.51e-24_F0115.88_h1h02.5e-24_l1h00,0,115.737778,115.937222,360,5760,1242778599,1253144799,1800,360,5760,1242778599,1253144799,1800,11.0,11.0_0
2,103_sqrtSX5.51e-24_F0115.88_h1h02.5e-24_l1h00,0,115.861111,116.060556,360,5760,1242347815,1252714015,1800,360,5760,1242347815,1252714015,1800,11.0,11.0_0
3,104_sqrtSX5.51e-24_F0115.88_h1h02.5e-24_l1h00,0,115.861667,116.061111,360,5760,1241598571,1251964771,1800,360,5760,1241598571,1251964771,1800,11.0,11.0_0
4,105_sqrtSX8.43e-24_F082_h1h02.28e-24_l1h00,0,81.906111,82.105556,360,5760,1244742292,1255108492,1800,360,5760,1244742292,1255108492,1800,8.0,8.0_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,95_sqrtSX6.95e-24_F082.45_h1h02.68e-24_l1h00,0,82.397222,82.596667,360,5760,1241103981,1251470181,1800,360,5760,1241103981,1251470181,1800,8.0,8.0_0
146,96_sqrtSX8.74e-24_F082.45_h1h03.36e-24_l1h00,0,82.275000,82.474444,360,5760,1245715266,1256081466,1800,360,5760,1245715266,1256081466,1800,8.0,8.0_0
147,97_sqrtSX8.74e-24_F082.45_h1h03.36e-24_l1h00,0,82.422778,82.622222,360,5760,1240074002,1250440202,1800,360,5760,1240074002,1250440202,1800,8.0,8.0_0
148,98_sqrtSX8.74e-24_F082.45_h1h03.36e-24_l1h00,0,82.353889,82.553333,360,5760,1243727722,1254093922,1800,360,5760,1243727722,1254093922,1800,8.0,8.0_0


# Check

In [14]:
os.makedirs(f"{OUTDIR}/spec_image_sample", exist_ok=True)

h_paths = sorted(glob.glob(f"{OUTDIR}/*h1h0*.hdf5"))#[:5]
for ii, h_path in enumerate(h_paths):
    file_id = Path(h_path).stem

    freq, h1_a, l1_a, h1_t, l1_t = load_hdf5(h_path)
    print(freq.shape)
    print(h1_a.shape, l1_a.shape)
    print(h1_t.shape, l1_t.shape)
    print("h1_t:", h1_t)
    print("l1_t:", l1_t)
    print("h1_t[0] - l1_t[0]:", h1_t[0] - l1_t[0])

    h1_ts_gaps = h1_t[1:] - h1_t[:-1] - 1800
    print("h1 sum gap:",  round(sum(h1_ts_gaps) / (24*3600), 2) , "days")
    #print(list(h1_ts_gaps))

    l1_ts_gaps = l1_t[1:] - l1_t[:-1] - 1800
    print("l1 sum gap:",  round(sum(l1_ts_gaps) / (24*3600), 2) , "days")
    #print(list(l1_ts_gaps))

    fig_h1, axs = tutorial_utils.plot_real_imag_spectrograms(h1_t, freq, h1_a)
    fig_l1, axs = tutorial_utils.plot_real_imag_spectrograms(l1_t, freq, l1_a)
    
    fig_h1.savefig(f"{OUTDIR}/spec_image_sample/{file_id}_fig_h1.png")
    fig_l1.savefig(f"{OUTDIR}/spec_image_sample/{file_id}_fig_l1.png")
    
    file_id, f, ts_h1, ts_l1, a_h1, a_l1, p_h1, p_l1 = get_hdf5(h_path, 0, 
                                                                is_p_mean=True,
                                                                is_show=True,
                                                                save_dir=f"{OUTDIR}/spec_image_sample",
                                                               )
    print(p_h1.max(), p_l1.max())
    
if DEBUG == False:
    from IPython.display import clear_output  # セルの出力削除（ログあると重すぎるので消す)
    clear_output()